#### H20 IMPORT AND INITIALIZATION

In [29]:
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init() #connecting to an H2O cluster 

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.1+12-LTS-39, mixed mode, sharing)
  Starting server from C:\Users\nidhi\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\nidhi\AppData\Local\Temp\tmpj7ni1vcs
  JVM stdout: C:\Users\nidhi\AppData\Local\Temp\tmpj7ni1vcs\h2o_nidhi_started_from_python.out
  JVM stderr: C:\Users\nidhi\AppData\Local\Temp\tmpj7ni1vcs\h2o_nidhi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_nidhi_c9923r
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.840 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [30]:
import pandas as pd

#### READ CSV FILE AND CONVERT TO h2o FRAME

In [35]:
df=pd.read_csv("s3://cis8395/final_dataset.csv")

#### DROP UNNECESSARY COLUMNS 

In [36]:
df.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Meal', 'Country',
       'MarketSegment', 'DistributionChannel', 'IsRepeatedGuest',
       'PreviousCancellations', 'PreviousBookingsNotCanceled',
       'ReservedRoomType', 'AssignedRoomType', 'BookingChanges', 'DepositType',
       'Agent', 'Company', 'DaysInWaitingList', 'CustomerType', 'ADR',
       'RequiredCarParkingSpaces', 'TotalOfSpecialRequests', 'Hotel',
       'Guests'],
      dtype='object')

In [37]:
df.drop(columns = ['ArrivalDateYear','ArrivalDateWeekNumber','MarketSegment', 'DistributionChannel', 'CustomerType', 
                   'AssignedRoomType','Company','ADR', 'Guests'], inplace = True)

#### CONVERT TO h2o DataFrame

In [38]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [39]:
h2o_df.types

{'IsCanceled': 'int',
 'LeadTime': 'int',
 'ArrivalDateMonth': 'enum',
 'ArrivalDateDayOfMonth': 'int',
 'StaysInWeekendNights': 'int',
 'StaysInWeekNights': 'int',
 'Meal': 'enum',
 'Country': 'enum',
 'IsRepeatedGuest': 'int',
 'PreviousCancellations': 'int',
 'PreviousBookingsNotCanceled': 'int',
 'ReservedRoomType': 'enum',
 'BookingChanges': 'int',
 'DepositType': 'enum',
 'Agent': 'enum',
 'DaysInWaitingList': 'int',
 'RequiredCarParkingSpaces': 'int',
 'TotalOfSpecialRequests': 'int',
 'Hotel': 'enum'}

#### TRAIN TEST SPLIT

In [40]:
train, test = h2o_df.split_frame(ratios=[0.8], seed = 1)

In [41]:
X = train.columns
y = 'IsCanceled'
X.remove(y)

train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [42]:
aml = H2OAutoML(max_models =20,balance_classes=True,seed =1)

aml.train(x=X, y=y, training_frame=train)

AutoML progress: |
16:59:52.98: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_3_20220422_165952

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0636546298708139
RMSE: 0.25229869177388514
LogLoss: 0.21997365292572998
Null degrees of freedom: 9893
Residual degrees of freedom: 9882
Null deviance: 11708.477684835332
Residual deviance: 4352.838644094346
AIC: 4376.838644094346
AUC: 0.979929011502889
AUCPR: 0.9497220133422051
Gini: 0.959858023005778

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5094478026440845: 


,,0,1,Error,Rate
0,0,6570.0,567.0,0.0794,(567.0/7137.0)
1,1,181.0,2576.0,0.0657,(181.0/2757.0)
2,Total,6751.0,3143.0,0.0756,(748.0/9894.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.509448,0.873220,180.0
1,max f2,0.426574,0.921815,205.0
2,max f0point5,0.695213,0.884164,119.0
3,max accuracy,0.551111,0.926218,166.0
4,max precision,0.999159,1.000000,0.0
5,max recall,0.126829,1.000000,323.0
6,max specificity,0.999159,1.000000,0.0
7,max absolute_mcc,0.509448,0.823260,180.0
8,max min_per_class_accuracy,0.523784,0.924758,176.0
9,max mean_per_class_accuracy,0.462456,0.930302,195.0



Gains/Lift Table: Avg response rate: 27.87 %, avg score: 32.95 %


...


Gains/Lift Table: Avg response rate: 27.59 %, avg score: 27.59 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010003,0.978554,3.624122,3.624122,1.000000,0.993190,1.000000,0.993190,0.036252,0.036252,262.412184,262.412184,0.036252
1,2,0.020006,0.946397,3.591955,3.608038,0.991124,0.962732,0.995562,0.977961,0.035931,0.072183,259.195508,260.803846,0.072061
2,3,0.030009,0.913343,3.479371,3.565149,0.960059,0.930136,0.983728,0.962019,0.034805,0.106988,247.937141,256.514945,0.106313
3,4,0.040012,0.887491,3.318538,3.503496,0.915680,0.899866,0.966716,0.946481,0.033196,0.140183,231.853760,250.349649,0.138344
4,5,0.050001,0.865428,3.242918,3.451442,0.894815,0.876481,0.952353,0.932498,0.032391,0.172575,224.291792,245.144247,0.169284
5,6,0.100001,0.769433,3.020280,3.235861,0.833383,0.815509,0.892868,0.874003,0.151016,0.323591,202.028029,223.586138,0.308795
6,7,0.150002,0.681500,2.603061,3.024928,0.718260,0.725889,0.834665,0.824632,0.130155,0.453746,160.306118,202.492798,0.419494
7,8,0.200003,0.588945,2.295241,2.842506,0.633323,0.636053,0.784330,0.777487,0.114764,0.568510,129.524142,184.250634,0.508937
8,9,0.300004,0.399996,1.773986,2.486333,0.489494,0.491414,0.686051,0.682130,0.177401,0.745911,77.398565,148.633278,0.615832
9,10,0.400006,0.260594,1.146012,2.151253,0.316218,0.327068,0.593593,0.593364,0.114603,0.860514,14.601189,115.125256,0.635998




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.830128,0.007166,0.825666,0.822326,0.841138,0.829606,0.831904
1,auc,0.904746,0.003155,0.905509,0.903003,0.905568,0.900611,0.909038
2,err,0.169872,0.007166,0.174334,0.177674,0.158862,0.170394,0.168096
3,err_count,2295.800000,93.726730,2361.000000,2405.000000,2161.000000,2278.000000,2274.000000
4,f0point5,0.684440,0.016659,0.679798,0.668883,0.710977,0.689088,0.673454
5,f1,0.717009,0.006668,0.722594,0.710903,0.725448,0.711207,0.714895
6,f2,0.753356,0.015194,0.771141,0.758555,0.740520,0.734793,0.761770
7,lift_top_group,3.625862,0.090861,3.555527,3.634801,3.577854,3.581302,3.779827
8,logloss,0.338922,0.007038,0.340236,0.340979,0.339228,0.346708,0.327457
9,max_per_class_error,0.219949,0.026836,0.192701,0.205961,0.249079,0.248594,0.203409



See the whole table with table.as_data_frame()


In [21]:
import pickle
pickle.dump(aml, open("autoMl.pkl", "wb"))

#### LEADERBOARD RESULTS

In [43]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_3_20220422_165952,0.904701,0.3389,0.790303,0.188761,0.331116,0.109638
StackedEnsemble_BestOfFamily_1_AutoML_3_20220422_165952,0.902198,0.34325,0.783221,0.188794,0.333554,0.111258
GBM_1_AutoML_3_20220422_165952,0.899928,0.348872,0.77839,0.192507,0.33633,0.113118
GBM_grid_1_AutoML_3_20220422_165952_model_4,0.899378,0.354353,0.779877,0.189789,0.338484,0.114572
GBM_4_AutoML_3_20220422_165952,0.899332,0.350336,0.778955,0.191712,0.336771,0.113415
DRF_1_AutoML_3_20220422_165952,0.897916,0.364573,0.770625,0.192962,0.340936,0.116238
GBM_3_AutoML_3_20220422_165952,0.897575,0.351209,0.773385,0.196276,0.337921,0.114191
GBM_2_AutoML_3_20220422_165952,0.897564,0.350755,0.772687,0.192976,0.337886,0.114167
GBM_5_AutoML_3_20220422_165952,0.896026,0.352738,0.768967,0.195212,0.339189,0.115049
GBM_grid_1_AutoML_3_20220422_165952_model_3,0.89554,0.354076,0.766967,0.194162,0.339731,0.115417


In [44]:
best_model=aml.leader
pred=best_model.predict(test)
prediction=pred.as_data_frame(use_pandas=True)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [45]:
prediction

,predict,p0,p1
0,0,0.771906,0.228094
1,0,0.840097,0.159903
2,0,0.996345,0.003655
3,0,0.996577,0.003423
4,1,0.277687,0.722313
...,...,...,...
16968,0,0.998548,0.001452
16969,0,0.809933,0.190067
16970,0,0.990086,0.009914
16971,1,0.551875,0.448125


In [46]:
best_model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.11471147260702182
RMSE: 0.33869082155709773
LogLoss: 0.35121385879307765
Null degrees of freedom: 16972
Residual degrees of freedom: 16961
Null deviance: 20101.93518840796
Residual deviance: 11922.305650589813
AIC: 11946.305650589813
AUC: 0.903498680252678
AUCPR: 0.7891816230501649
Gini: 0.806997360505356

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46777846650127364: 


,,0,1,Error,Rate
0,0,10415.0,1820.0,0.1488,(1820.0/12235.0)
1,1,1088.0,3650.0,0.2296,(1088.0/4738.0)
2,Total,11503.0,5470.0,0.1713,(2908.0/16973.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.467778,0.715125,191.0
1,max f2,0.235121,0.805841,276.0
2,max f0point5,0.669967,0.732260,119.0
3,max accuracy,0.608429,0.839804,140.0
4,max precision,0.999393,1.000000,0.0
5,max recall,0.000713,1.000000,398.0
6,max specificity,0.999393,1.000000,0.0
7,max absolute_mcc,0.490985,0.597717,182.0
8,max min_per_class_accuracy,0.411887,0.816167,212.0
9,max mean_per_class_accuracy,0.393443,0.817889,219.0



Gains/Lift Table: Avg response rate: 27.91 %, avg score: 32.41 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010016,0.985891,3.582313,3.582313,1.000000,0.995448,1.000000,0.995448,0.035880,0.035880,258.231321,258.231321,0.035880
1,2,0.020032,0.963764,3.519096,3.550705,0.982353,0.974900,0.991176,0.985174,0.035247,0.071127,251.909592,255.070457,0.070882
2,3,0.030048,0.939553,3.392661,3.498023,0.947059,0.950806,0.976471,0.973718,0.033981,0.105108,239.266134,249.802349,0.104127
3,4,0.040005,0.923639,3.370342,3.466244,0.940828,0.931535,0.967599,0.963219,0.033558,0.138666,237.034202,246.624415,0.136868
4,5,0.050021,0.907028,3.139792,3.400877,0.876471,0.914671,0.949352,0.953498,0.031448,0.170114,213.979217,240.087685,0.166599
5,6,0.100041,0.836399,2.962054,3.181465,0.826855,0.869903,0.888104,0.911700,0.148164,0.318278,196.205403,218.146544,0.302749
6,7,0.150003,0.769754,2.623369,2.995579,0.732311,0.804022,0.836214,0.875836,0.131068,0.449346,162.336852,199.557927,0.415263
7,8,0.200024,0.691660,2.274284,2.815202,0.634865,0.733587,0.785862,0.840263,0.113761,0.563107,127.428365,181.520225,0.503687
8,9,0.300006,0.507127,1.749993,2.460202,0.488509,0.598116,0.686764,0.759563,0.174968,0.738075,74.999272,146.020214,0.607711
9,10,0.399988,0.356003,1.182142,2.140734,0.329994,0.426918,0.597584,0.676414,0.118193,0.856268,18.214225,114.073423,0.632975


In [47]:
m = aml.get_best_model()
m

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_3_20220422_165952

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0636546298708139
RMSE: 0.25229869177388514
LogLoss: 0.21997365292572998
Null degrees of freedom: 9893
Residual degrees of freedom: 9882
Null deviance: 11708.477684835332
Residual deviance: 4352.838644094346
AIC: 4376.838644094346
AUC: 0.979929011502889
AUCPR: 0.9497220133422051
Gini: 0.959858023005778

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5094478026440845: 


,,0,1,Error,Rate
0,0,6570.0,567.0,0.0794,(567.0/7137.0)
1,1,181.0,2576.0,0.0657,(181.0/2757.0)
2,Total,6751.0,3143.0,0.0756,(748.0/9894.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.509448,0.873220,180.0
1,max f2,0.426574,0.921815,205.0
2,max f0point5,0.695213,0.884164,119.0
3,max accuracy,0.551111,0.926218,166.0
4,max precision,0.999159,1.000000,0.0
5,max recall,0.126829,1.000000,323.0
6,max specificity,0.999159,1.000000,0.0
7,max absolute_mcc,0.509448,0.823260,180.0
8,max min_per_class_accuracy,0.523784,0.924758,176.0
9,max mean_per_class_accuracy,0.462456,0.930302,195.0



Gains/Lift Table: Avg response rate: 27.87 %, avg score: 32.95 %


...


Gains/Lift Table: Avg response rate: 27.59 %, avg score: 27.59 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010003,0.978554,3.624122,3.624122,1.000000,0.993190,1.000000,0.993190,0.036252,0.036252,262.412184,262.412184,0.036252
1,2,0.020006,0.946397,3.591955,3.608038,0.991124,0.962732,0.995562,0.977961,0.035931,0.072183,259.195508,260.803846,0.072061
2,3,0.030009,0.913343,3.479371,3.565149,0.960059,0.930136,0.983728,0.962019,0.034805,0.106988,247.937141,256.514945,0.106313
3,4,0.040012,0.887491,3.318538,3.503496,0.915680,0.899866,0.966716,0.946481,0.033196,0.140183,231.853760,250.349649,0.138344
4,5,0.050001,0.865428,3.242918,3.451442,0.894815,0.876481,0.952353,0.932498,0.032391,0.172575,224.291792,245.144247,0.169284
5,6,0.100001,0.769433,3.020280,3.235861,0.833383,0.815509,0.892868,0.874003,0.151016,0.323591,202.028029,223.586138,0.308795
6,7,0.150002,0.681500,2.603061,3.024928,0.718260,0.725889,0.834665,0.824632,0.130155,0.453746,160.306118,202.492798,0.419494
7,8,0.200003,0.588945,2.295241,2.842506,0.633323,0.636053,0.784330,0.777487,0.114764,0.568510,129.524142,184.250634,0.508937
8,9,0.300004,0.399996,1.773986,2.486333,0.489494,0.491414,0.686051,0.682130,0.177401,0.745911,77.398565,148.633278,0.615832
9,10,0.400006,0.260594,1.146012,2.151253,0.316218,0.327068,0.593593,0.593364,0.114603,0.860514,14.601189,115.125256,0.635998




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.830128,0.007166,0.825666,0.822326,0.841138,0.829606,0.831904
1,auc,0.904746,0.003155,0.905509,0.903003,0.905568,0.900611,0.909038
2,err,0.169872,0.007166,0.174334,0.177674,0.158862,0.170394,0.168096
3,err_count,2295.800000,93.726730,2361.000000,2405.000000,2161.000000,2278.000000,2274.000000
4,f0point5,0.684440,0.016659,0.679798,0.668883,0.710977,0.689088,0.673454
5,f1,0.717009,0.006668,0.722594,0.710903,0.725448,0.711207,0.714895
6,f2,0.753356,0.015194,0.771141,0.758555,0.740520,0.734793,0.761770
7,lift_top_group,3.625862,0.090861,3.555527,3.634801,3.577854,3.581302,3.779827
8,logloss,0.338922,0.007038,0.340236,0.340979,0.339228,0.346708,0.327457
9,max_per_class_error,0.219949,0.026836,0.192701,0.205961,0.249079,0.248594,0.203409



See the whole table with table.as_data_frame()


In [ ]:
aml.leader.varimp()
model = h2o.get_model("StackedEnsemble_AllModels_1_AutoML_3_20220422_165952")
model.varimp_plot(num_of_features=8)